In [1]:
!pip freeze | grep scikit-learn

scikit-learn @ file:///croot/scikit-learn_1680198742267/work


In [12]:
import pickle
import pandas as pd
import numpy as np

In [17]:
year = 2022
month = 2

In [3]:
with open('model.bin', 'rb') as f_in:
    dv, model = pickle.load(f_in)

In [4]:
categorical = ['PULocationID', 'DOLocationID']

def read_data(filename):
    df = pd.read_parquet(filename)
    
    df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
    df['duration'] = df.duration.dt.total_seconds() / 60

    df = df[(df.duration >= 1) & (df.duration <= 60)].copy()

    df[categorical] = df[categorical].fillna(-1).astype('int').astype('str')
    
    return df

In [5]:
df = read_data('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2022-02.parquet')

In [6]:
dicts = df[categorical].to_dict(orient='records')
X_val = dv.transform(dicts)
y_pred = model.predict(X_val)

In [15]:
df.head(2)

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee,duration
0,1,2022-02-01 00:06:58,2022-02-01 00:19:24,1.0,5.4,1.0,N,138,252,1,17.0,1.75,0.5,3.9,0.00,0.3,23.45,0.0,1.25,12.433333
1,1,2022-02-01 00:38:22,2022-02-01 00:55:55,1.0,6.4,1.0,N,138,41,2,21.0,1.75,0.5,0.0,6.55,0.3,30.10,0.0,1.25,17.550000


In [11]:
type(y_pred)

numpy.ndarray

In [13]:
#Q1.

std_dev = np.std(y_pred)
# print the result
print("Standard deviation:", std_dev)

Standard deviation: 5.28140357655334


In [18]:
df['ride_id'] = f'{year:04d}/{month:02d}_' + df.index.astype('str')

In [20]:
df_result = pd.DataFrame({
    'ride_id': df['ride_id'],
    'prediction': y_pred
})

In [22]:
# define the output file name and location
output_file = 'results.parquet'

df_result.to_parquet(
    output_file,
    engine='pyarrow',
    compression=None,
    index=False
)